# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846821079182                   -0.70    4.0
  2   -7.852314179360       -2.26       -1.53    1.0
  3   -7.852646031314       -3.48       -2.52    3.2
  4   -7.852646675565       -6.19       -3.33    2.2
  5   -7.852646685806       -7.99       -4.73    1.2
  6   -7.852646686727       -9.04       -5.21    3.8
  7   -7.852646686729      -11.68       -5.84    1.0
  8   -7.852646686730      -12.40       -7.40    2.0
  9   -7.852646686730   +  -15.05       -7.66    3.0
 10   -7.852646686730      -15.05       -8.21    1.2
 11   -7.852646686730   +  -14.57       -9.53    2.0
 12   -7.852646686730      -14.27      -10.37    2.2
 13   -7.852646686730   +  -14.57      -11.04    2.5
 14   -7.852646686730   +    -Inf      -12.06    2.2


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846787376778                   -0.70           4.0
  2   -7.852527124566       -2.24       -1.63   0.80    2.2
  3   -7.852637422954       -3.96       -2.69   0.80    1.0
  4   -7.852646291416       -5.05       -3.28   0.80    2.0
  5   -7.852646677676       -6.41       -4.05   0.80    1.8
  6   -7.852646686443       -8.06       -4.70   0.80    1.5
  7   -7.852646686714       -9.57       -5.86   0.80    1.5
  8   -7.852646686730      -10.80       -6.44   0.80    2.5
  9   -7.852646686730      -12.92       -7.18   0.80    1.8
 10   -7.852646686730      -14.27       -8.66   0.80    1.8
 11   -7.852646686730      -15.05       -9.26   0.80    3.0
 12   -7.852646686730      -15.05       -9.85   0.80    1.5
 13   -7.852646686730   +  -14.75      -10.21   0.80    1.8
 14   -7.852646686730      -15.05      -10.82   0.80    1.2
 15   -7.852646686730      -14.57      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.414042e+01     3.682112e+00
 * time: 0.4008371829986572
     1     8.360356e-01     2.027574e+00
 * time: 0.6151490211486816
     2    -1.728534e+00     2.343325e+00
 * time: 0.6388571262359619
     3    -3.721390e+00     2.062259e+00
 * time: 0.672370195388794
     4    -4.903412e+00     1.806843e+00
 * time: 0.7056190967559814
     5    -6.631667e+00     1.298201e+00
 * time: 0.7386970520019531
     6    -7.343751e+00     8.954196e-01
 * time: 0.7720551490783691
     7    -7.573097e+00     8.837704e-01
 * time: 0.7951211929321289
     8    -7.728419e+00     3.159068e-01
 * time: 0.8182351589202881
     9    -7.787883e+00     1.064033e-01
 * time: 0.8415250778198242
    10    -7.827409e+00     1.213519e-01
 * time: 0.864922046661377
    11    -7.842887e+00     6.554982e-02
 * time: 0.8882231712341309
    12    -7.848649e+00     3.734445e-02
 * time: 0.9116110801696777
    13    -7.850968e+00     3.023308e-02
 * time: 0.934986114501

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846805540033                   -0.70    3.8
  2   -7.852294777133       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686228                   -2.55
  2   -7.852646686730       -9.30       -5.74
  3   -7.852646686730   +    -Inf      -11.07
  4   -7.852646686730   +    -Inf      -16.00


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 7.848417280441665e-13
|ρ_newton - ρ_scfv| = 9.276274017891365e-14
|ρ_newton - ρ_dm|   = 1.3732801838127205e-9
